In [1]:
import pandas as pd
import sys, os
import numpy as np
import json
import shapefile

## Read shapefile

In [2]:
sf_path = os.path.join('..', 'vehicle_based_heatmap', 'new_link', 'new_link_latlon')
link_sf = shapefile.Reader(sf_path)
shapeRecs = link_sf.shapeRecords()

link_map = {}
for data in shapeRecs:
    link_map[data.record[1]] = {}
    link_map[data.record[1]]['points'] = []
    for _item in data.shape.points:
        link_map[data.record[1]]['points'].append([round(_item[0], 5),  round(_item[1], 5)])
    link_map[data.record[1]]['ffs'] = data.record[4]
    link_map[data.record[1]]['lanes'] = data.record[5]
    link_map[data.record[1]]['length'] = data.record[6]
    link_map[data.record[1]]['isOD'] = data.record[7]
#     print(data.record)
#     print(link_map)
#     break

## Read link congestion file && process to geojson format

In [11]:
vehloc_path = os.path.join('..', '..', '..', 'data', 'input_files_MckeesRocks_SPC', 'link_cong', 'link_cong_raw.txt')

# create a new python dict to contain our geojson data, using geojson format
geojson_car = {'type':'FeatureCollection', 'features':[]}
geojson_truck = {'type':'FeatureCollection', 'features':[]}

with open(vehloc_path, 'r') as infile:
    lines = infile.readlines()
    for line in lines:
        data = line.rstrip().split(' ')
        _link_ID = data[0]
        _interval = int(data[1])//180
        if (link_map[_link_ID]['isOD'] == '0') and (_interval % 2 == 0):
            _interval //= 2
            if _interval < 9:
                _car_flow = float(data[2])
                _truck_flow = float(data[3])
                _car_fft = float(data[6])
                _truck_fft = float(data[7])
                _car_cong = float(data[4])/_car_fft
                _truck_cong = float(data[5])/_truck_fft
                if data[8] == 'inf':
                    _car_speed = float(link_map[_link_ID]['ffs'])
                else:
                    _car_speed = float(data[8])
                if data[9] == 'inf':
                    _truck_speed = float(link_map[_link_ID]['ffs']) * 0.8
                else:
                    _truck_speed = float(data[9])

                # create a feature template to fill in
                feature_car = {'type':'Feature',
                               'properties':{},
                               'geometry':{'type': 'LineString',
                                           'coordinates':[]
                                }}
                feature_truck = {'type':'Feature',
                               'properties':{},
                               'geometry':{'type': 'LineString',
                                           'coordinates':[]
                                }}

                for i in range(len(link_map[_link_ID]['points'])):
                    feature_car['geometry']['coordinates'].append(link_map[_link_ID]['points'][i])
                    feature_truck['geometry']['coordinates'].append(link_map[_link_ID]['points'][i])

                feature_car['properties']['ID'] = _link_ID
                feature_car['properties']['ffs_car'] = link_map[_link_ID]['ffs']
                feature_car['properties']['lanes'] = link_map[_link_ID]['lanes']
                feature_car['properties']['length'] = link_map[_link_ID]['length']
                feature_car['properties']['time_point'] = _interval
                feature_car['properties']['car_flow'] = _car_flow
                feature_car['properties']['car_fft'] = _car_fft
                feature_car['properties']['car_cong'] = _car_cong
                feature_car['properties']['car_speed'] = _car_speed

                feature_truck['properties']['ID'] = _link_ID
                feature_truck['properties']['ffs_car'] = link_map[_link_ID]['ffs']
                feature_truck['properties']['lanes'] = link_map[_link_ID]['lanes']
                feature_truck['properties']['length'] = link_map[_link_ID]['length']
                feature_truck['properties']['time_point'] = _interval
                feature_truck['properties']['truck_flow'] = _truck_flow
                feature_truck['properties']['truck_fft'] = _truck_fft
                feature_truck['properties']['truck_cong'] = _truck_cong
                feature_truck['properties']['truck_speed'] = _truck_speed

                # add this feature to the list of features inside our dict
                geojson_car['features'].append(feature_car)
                geojson_truck['features'].append(feature_truck)

geojson_str_car = json.dumps(geojson_car, indent=2)
geojson_str_truck = json.dumps(geojson_truck, indent=2)

# save the geojson result to a file
output_filename_car = 'link_cong_car.geojson'
with open(output_filename_car, 'w') as output_file_car:
    output_file_car.write('var data_cong_car = {};'.format(geojson_str_car))
output_filename_truck = 'link_cong_truck.geojson'
with open(output_filename_truck, 'w') as output_file_truck:
    output_file_truck.write('var data_cong_truck = {};'.format(geojson_str_truck))

# how many features did we save to the geojson file?
print('{} geotagged features saved to car file'.format(len(geojson_car['features'])))
print('{} geotagged features saved to truck file'.format(len(geojson_truck['features'])))

132552 geotagged features saved to car file
132552 geotagged features saved to truck file
